# Chain level Steenrod operations

### Abstract
In this notebook we implement functions that construct representatives for Steenrod operations acting on algebras over the Barratt-Eccles and Surjection $E_\infty$-operads. An important example we treat in detail is the case of cochains of simplicial sets. 

## Introduction <a name="introduction"></a>
...


# Table of contents
1. [Introduction](#introduction)
2. [Preliminaries](#preliminaries)
    1. [Free modules](#free_modules)
    2. [Group rings](#group_rings)
        1. [Cyclic group](#cyclic_groups)
        2. [Symmetric group](#symmetric_groups)
3. [Coskeletal chain complexes](#dg_modules)
    1. [Cyclic version](#cyclic_dg_modules)
        1. [The map $\psi$](#psi)
4. [Operads](#operads)
    1. [Barratt-Eccles](#barrat_eccles)
        1. [The map $\phi$](#phi)
    2. [Surjection](#surjection)
        1. [Table reduction morphism](#table_reduction)
    3. [Eilenberg-Zilber](#eilenberg_zilber)
        1. [Interval cut morphism](#interval_cut)
5. [Power operations](#power_operations)
    1. [Chain convention](#chain_convention)
    2. [Cochain convention](#cochain_convention)
6. [References](#references)

## Preliminaries <a name="preliminaries"></a>

### Free modules <a name="free_modules"></a>

**Definition**. Let $R$ be a ring and $B$ a set. The free $R$-module generated by $B$ consists of all $R$-linear combination of elements in $B$

\begin{equation*}
R[B] = \Big\{ \sum_i r_ib_i\ |\ r_i \in R, b_i \in B \Big\}.
\end{equation*}

**Implementation**. For a ring $R$ equal to $\mathbb Z$ or $\mathbb Z/n\mathbb Z$ we introduce a class modeling elements in free $R$-modules. The ring is specified via the class attribute `torsion`, where `'free'` sets $R = \mathbb Z$ and a positive `int` sets $R = \mathbb Z/n\mathbb Z$. The class supports `+` addition, `-` substraction and `*` scaling.

In [1]:
from clesto import Module_element

# The initialization takes a dict with int values
x = Module_element({'a':1, 'b':2, 'c':3})

print(f'If x = {x} then \n' +
      f'-x = {-x} \n' + 
      f'x+x = {x+x} \n' + 
      f'2*x = {2*x} \n' + 
      f'x-x = {x-x}')

If x = a+2b+3c then 
-x = -a-2b-3c 
x+x = 2a+4b+6c 
2*x = 2a+4b+6c 
x-x = 0


In [2]:
# The attribute torsion can be set in three ways:
# a) For all future instances using the class attribute default_torsion.
# b) Using a flag at initialization of an instance.
# c) Using a setter method on an initialized instance. Directly changing 
#    the attribute is not adviced.

Module_element.default_torsion = 'free'
x = Module_element({'a':1, 'b':2, 'c':3})
print(f'x = {x} with torsion = {x.torsion}')
y = Module_element(x, torsion=3)
print(f'x = {y} with torsion = {y.torsion}')
x.set_torsion(2)
print(f'x = {x} with torsion = {x.torsion}')

x = a+2b+3c with torsion = free
x = a+2b with torsion = 3
x = a+c with torsion = 2


### Group rings <a name="group_rings"></a>

**Definition**. Let $R$ be a ring and $\Gamma$ a group. The free $R$-module generated by $\Gamma$ is a ring with the induced product

\begin{equation*}
\left( \sum_i r_ia_i \right) \left( \sum_j s_jb_j \right) = \sum_{i,j} (r_is_j)(a_ib_{j}).
\end{equation*}

#### Cyclic groups <a name="cyclic_groups"></a>

**Definition**. Let $C = \langle \rho \rangle$ be the infinite cyclic group and $C_r = \langle \rho\ |\ \rho^r = 1 \rangle$ be the cyclic group of order $r$.

**Implementation**. We introduce a subclass of `Module_element` modeling elements in the group ring of cyclic groups when the base ring is $\mathbb Z$ or $\mathbb Z/n\mathbb Z$. It inherents the `torsion` attribute, as well as the addition `+`, substraction `-`, and scaling `*` methods; additionally, it has an `order` attribute and supports the ring product `*`.

In [3]:
from clesto import CyclicModule_element

x = CyclicModule_element({0:1, 1:1, 2:1})
print(f'If x = {x} then x*x = {x*x}')

ImportError: cannot import name 'CyclicModule_element' from 'clesto' (c:\users\aniba\github\clesto\clesto\__init__.py)

In [ ]:
# Like the attribute torsion, the attribute order can be set in three ways:
# a) For all future instances using the class attribute default_order.
# b) Using a flag at initialization of an instance.
# c) Using a setter method on an initialized instance. Directly changing 
#    the attribute is not adviced.

Module_element.default_order = 'infinite'
x = CyclicModule_element({0:1, 3:1, 6:1})
print(f'x = {x} with torsion = {x.order}')
y = CyclicModule_element(x, order=3)
print(f'x = {y} with torsion = {y.order}')
x.set_order(2)
print(f'x = {x} with torsion = {x.order}')

There are two important elements in $\mathbb Z/n\mathbb Z[C_n]$

\begin{equation*}
T = \rho-1 \quad \text{ and } \quad N = 1+\rho+\cdots+\rho^{n-1}
\end{equation*}

defining a minimal resolution

\begin{equation*}
W: \qquad \mathbb Z/n\mathbb Z[C_n] \stackrel{T}{\longleftarrow} \mathbb Z/n\mathbb Z[C_n] \stackrel{N}{\longleftarrow} \mathbb Z/n\mathbb Z[C_n] \stackrel{T}{\longleftarrow} \cdots
\end{equation*}

of $\mathbb Z/n\mathbb Z$ by a free differential graded $\mathbb Z/n\mathbb Z[C_n]$-module. We denote a chosen generator in degree $d$ by $e_d$.

**Implementation**. The class `CyclicModule_element` contains the methods `transposition_element` and `norm_element` returning models for $T$ and $N$ given an `int`. 

In [ ]:
n = 4
T = CyclicModule_element.transposition_element(n)
N = CyclicModule_element.norm_element(n)
print(f'For Z/{n}Z[C_{n}] the special elements are \n' +
      f'T = {T} and N = {N}. \n' +
      f'They satisfy T*N = {T*N} and N*T = {N*T}')

#### Symmetric groups <a name="symmetric_groups"></a>

**Definition**. Let $\Sigma_r$ denote the group of permutations of $r$ elements. More explicitly, a permutation is a bijection of set $\{1,\dots,r\}$ and $r$ is referred to as the arity of the permutation. A permutation $\pi$ can be represented by the tuple of its images $(\pi(1), \dots, \pi(r))$.

**Implementation**. We introduce a subclass of `Module_element` modeling elements in $\mathbb Z[\Sigma_r]$ and $\mathbb Z/n\mathbb Z[\Sigma_r]$. It inherents addition `+`, substraction `-`, and scaling `*` and, additionally, it supports the ring product `*`.

In [ ]:
from clesto import SymmetricModule_element

x = SymmetricModule_element({(2,3,1):5}, torsion=3)
print(f'If x = {x} its arity is {x.arity} and \nx*x = {x*x} and x*x*x = {x*x*x}')

#### Composition (TO DO: description)

For all integers $r, s \geq 1$ and $k \in \{1, \ldots, s\}$ we define a partial composition
\begin{equation*}
\circ_k : \Sigma_r \times \Sigma_s \to \Sigma_{r+s-1} \, .
\end{equation*}

In [ ]:
from clesto import SymmetricModule_element

x = SymmetricModule_element({(3,2,1):1})
y = SymmetricModule_element({(1,3,2):1})
print(f'If x = {x} and y = {y}\nthen x o_2 y = {x.compose(y,2)}')

We also define a composition
\begin{equation*}
	\circ_{\Sigma} : \Sigma_r \times \Sigma_{s_1} \times \cdots \times \Sigma_{s_r} \ \to \
	\Sigma_{s_1 + \cdots + s_{r}} \, .
\end{equation*}

In [ ]:
from clesto import SymmetricModule_element

x = SymmetricModule_element({(3,2,1):1})
y1 = SymmetricModule_element({(1,2):1})
y2 = SymmetricModule_element({(1,3,4,5,2):1})
y3 = SymmetricModule_element({(4,1,3,2):1})
print(f'x = {x}\ny1 = {y1}\ny2 = {y2}\ny3 = {y3}\nx o (y1,y2,y3) = {x.compose(y1,y2,y3)}')

## Coskeletal chain complexes <a name="dg_modules"></a>

**Definition**. Let $R$ be a ring and $B$ a set. The coskeletal differential graded $R$-module generated by $B$ is the differential graded $R$-module

\begin{equation*}
N_d(B;R) = \frac{R \big\{ (b_0,\dots,b_d)\ |\ b_i \in B \big\}}{R \{b_i = b_{i+1} \}}
\qquad
\partial(b_0,\dots,b_d) = \sum_{i=1}^n (-1)^i(b_0,\dots,\widehat{b}_i,\dots,b_d).
\end{equation*}

**Remark 1**. This terminology is motivated by the fact that $N_\bullet(B;R)$ is isomorphic to the normalized chains of the 0-coskeletal simplicial set generated by $B$.

**Implementation**. We introduce a subclass of `Module_element` that models elements in $N_\bullet(B;\mathbb Z)$ and $N_\bullet(B;\mathbb Z/n\mathbb Z)$. It inherents addition `+`, substraction `-`, and scaling `*`, additionally, it contains a method `boundary` modeling $\partial$.

In [ ]:
from clesto import DGModule_element

# It automatically removes degenerate simplices
x = DGModule_element({('a','b','c'):1, ('a','a','c'):2, ('a','b','a'):2})
print(f'Let x = {x} then\nboundary(x) = {x.boundary()}')

### Cyclic version <a name="cyclic_dg_modules"></a> 
 
**Definition**. Let $R$ be a ring and $\gamma$ a group. The coskeletal differential graded $R$-module generated by $\Gamma$ is naturally a left $R[\Gamma]$-module via

\begin{equation*}
\gamma (\gamma_0, \dots, \gamma_d) = (\gamma \gamma_0, \dots, \gamma \gamma_d).
\end{equation*}

**Implementation**. We introduce a subclass of `DGModule_element` that models elements in $N_\bullet(C_n;\mathbb Z/m\mathbb Z)$ as well as its torsion-free and/or infinite order versions. It inherents addition `+`, substraction `-`, scaling `*`, and the `boundary` method, additionally, it supports the action `*` of the group ring.

In [ ]:
from clesto import CyclicDGModule_element

n = m = 3
x = CyclicDGModule_element({(0,1,0,4):1}, torsion=n, order=m)
N = CyclicModule_element.transposition_element(n)
print(f'Let x = {x} and N = {N} then \n' + 
      f'Nx = {N(x)} and \n' +
      f'boundary(x) = {x.boundary()}')

#### The map $\psi$ <a name="psi"></a>

**Definition**. The quasi-isomorphism $\psi : W \to N_\bullet(C_p;\mathbb Z/p\mathbb Z)$ is defined by

\begin{align*}
\psi(e_{2k}) & = 
\sum_{i=1}^{k} \sum_{r_i=1}^{p-1}
\big( 1, \rho^{I_1(r_1,\dots,r_k)}, \dots,\ \rho^{I_{2k}(r_1,\dots,r_k)} \big) \\
\psi(e_{2k+1}) & = 
\sum_{i=1}^{k} \sum_{r_i=1}^{p-1}
\big( 1, \rho, \rho^{I_1(r_1,\dots,r_k)+1}, \dots,\ \rho^{I_{2k}(r_1,\dots,r_k)+1} \big)
\end{align*}
where
\begin{equation*}
I_s(r_1,\dots,r_k) =
r_1 + \cdots + r_{\lceil s/2 \rceil} + \lfloor s/2 \rfloor
\end{equation*}

with $\lceil - \rceil$ and $\lfloor - \rfloor$ denoting the integer ceiling and floor functions.

**Implementation**. The class `CyclicModule_element` contains a method `psi` modeling this map.

In [ ]:
n = 3
N = CyclicModule_element.norm_element(3)

print(f'Let us consider the norm element for Z/{n}Z[C_{n}]\n'+
      f'in degree 2: N = {N} then \n'+
      f'psi(N) = {N.psi(2)}')

## Operads <a name="operads"></a>

**Definition**. Let $R$ be a ring. An operad $\mathcal O$ is a set $\big\{ \mathcal O(r) \big\}_{r \in \mathbb N}$ of differential graded left $R[\Sigma_r]$-modules and morphisms

\begin{equation*}
\circ_{\mathcal O} : \mathcal O(r) \otimes \mathcal O(s_1) \otimes \cdots \otimes \mathcal O(s_r) \to \mathcal O(s_1+\cdots+s_r)
\end{equation*}

satisfying: associativity, unitality and equivariance. 

PARTIAL COMPOSITION

The operad $\mathcal O$ is said to be an $E_\infty$-operad if $O(r)$ is a free $R[\Sigma_r]$-module resolution of $R$.

An algebra over an operad $\mathcal O$ is a collection of maps

\begin{equation*}
\mathcal O(r) \otimes A^{\otimes r} \to A
\end{equation*}

compatible with the operadic structure. This structure is equivalent to an operad map 

\begin{equation*}
\mathcal O \to \mathrm{End}_{A}
\end{equation*}

where the operad
\begin{equation*}
\mathrm{End}_{A} = \big\{ \mathrm{End}(A^{\otimes r}, A) \big\}_{r \geq 0}
\end{equation*}

has composition structure induced from the usual composition of linear maps and the group action is induced from permutations of factors in the domain.

### Barratt-Eccles operad $\mathcal E$ <a name="barrat_eccles"></a>

**Definition**. Let $R$ be a ring. The arity $r$ part $\mathcal E(r)$ of the Barratt-Eccles operad $\mathcal E$ is the coskeletal differential graded $R[\Sigma_r]$-module generated by $\Sigma_r$. Its composition is best described using Remark 1. 

DESCRIBE

**Implementation**. We introduce a subclass of `DGModule_element` that models elements in the Barrat-Eccles operad when $R = \mathbb Z$ or $R = \mathbb Z/n\mathbb Z$. It inherents addition `+`, substraction `-`, scaling `*`, and `boundary` methods, additionally, it supports the left action of the correponding symmetric group ring `*`, and it contains the method `compose` modeling the operadic composition.

In [ ]:
from clesto import BarrattEccles_element

b = BarrattEccles_element({((1,2,3),(3,2,1)):1})
x = SymmetricModule_element({(1,3,2):1})

print(f'Let b = {b} and x = {x} then \n' + 
      f'x(b) = {x(b)} and \n' +
      f'boundary(b) = {b.boundary()}')

#### Composition (TO DO: description)

For all integers $r, s, d, e \geq 1$ and every $k \in \{1, \ldots, s\}$ we define a partial composition

\begin{equation*}
\circ_k : \mathcal E(r)_d \otimes \mathcal E(s)_e \to \mathcal E(r + s - 1)_{d+e} \, .
\end{equation*}

In [ ]:
from clesto import BarrattEccles_element

b1 = BarrattEccles_element({((1,2),(2,1)):1, ((2,1),(1,2)):2})
b2 = BarrattEccles_element({((2,1,3),(2,3,1)):3})

print(f'b1 = {b1}\nb2 = {b2}\nb1 o_2 b2 = {b1.compose(b2, 2)}')

We also define a composition
\begin{equation*}
\circ : \mathcal E(r)_d \otimes \mathcal E(s_1)_d \otimes \cdots \otimes \mathcal E(s_r)_d \to \mathcal E(s_1 + \cdots + s_r)_{(r+1)d} \, .
\end{equation*}

In [ ]:
from clesto import BarrattEccles_element

b = BarrattEccles_element({((1,2,3),(1,3,2)):1})
c1 = BarrattEccles_element({((2,1),(1,2)):1})
c2 = BarrattEccles_element({((1,2,3),):2})
c3 = BarrattEccles_element({((2,1),):3})

print(f'For b  = {b} \n' +
      f'c1 = {c1} \n' +
      f'c2 = {c2} \n' + 
      f'c3 = {c3} \nwe have\n' +
      f'b o (c1,c2,c3) = \n{b.compose(c1,c2,c3)}')

#### The map $\phi$ <a name="phi"></a>

**Definition**. The generator $\rho$ of $C_p$ can be regaded as the permutation

\begin{equation*}
\rho(1,2,\dots,p) = (2,\dots,p,1)
\end{equation*}

which induces an inclusion of associated coskeletal differential graded modules

\begin{equation*}
\phi: N_\bullet(C_p;\mathbb Z/p\mathbb Z) \to N_\bullet(\Sigma_p;\mathbb Z/p\mathbb Z) = \mathcal E(p)
\end{equation*}

**Implementation**. The class `CyclicDGModule_element` contains a method `phi` modeling this map.

In [ ]:
x = CyclicDGModule_element({(0,5,0,5):1}, torsion=4, order=4)

print(f'Let x = {x} then \nphi(x) = {x.phi()}')

### Surjection operad <a name="surjection"></a>

**Definition** Let $R$ be a ring. The arity $r$ part $\mathcal Surj(r)$ of the Surjection operad $Surj$ is the coskeletal differential graded $R$-module generated by $\{1,\dots,r\}$ modulo tuples defining functions $\{1,\dots,r+d\} \to \{1,\dots,r\}$ that fail to be surjective. The symmetric group action is induced for any $\sigma \in \Sigma_r$ by

\begin{equation*}
\sigma (s(1), \dots, s(r+s)) = (\sigma s(1), \dots, \sigma s(r+s))
\end{equation*}

DESCRIBE COMPOSITION

**Implementation**. We introduce a subclass of `DGModule_element` that models elements in the Surjection operad when $R = \mathbb Z$ or $R = \mathbb Z/n\mathbb Z$. It inherents addition `+`, substraction `-`, scaling `*`, and `boundary` method, additionally, it supports the left action of the correponding symmetric group ring `*`, and it contains a method `compose` modeling the operadic composition.

IMPLEMENT COMPOSITION

In [ ]:
from clesto import Surjection_element

s = Surjection_element({(1,2,3,1):1, (2,3,2,1):2, (2,3,2,3):1})
x = SymmetricModule_element({(2,1,3):2})

print(f'Let s = {s} \nand x = {x} then \nx(s) = {x(s)}')

#### Table reduction morphism <a name="table_reduction"></a>

**Definition**. The table reduction morphism 
\begin{equation*}
TR : \mathcal E \to Surj
\end{equation*}
is a quasi-ismomorphism whose definition is presented in [1].

**Implementation**. The class `BarrattEccles_element` contains the method `table_reduction` modeling this map.

In [ ]:
b = BarrattEccles_element({((1,2,3),(1,3,2)):1})
print(f'If b = {b} then TR(b) = {b.table_reduction()}')

### Eilenberg-Zilber operad <a name="eilenberg_zilber"></a>

**Definition** Operad $\mathrm{End}(N_\bullet)$ of natural transformations of the functor of normalized chains.

**Lemma** A basis given by pairs $(op, n)$ where $op = d_{U_1} \otimes \cdots \otimes d_{U_r}$ and $n$ is the degree of the domain.

**Implementation**. 

In [ ]:
from clesto import EilenbergZilber_element

x = EilenbergZilber_element({((1,2), (2,1,2), tuple()):2,
                              ((1,2), (1,1,2), (1,)):3})
print(x)

### $IC : Surj \to \mathrm{End}(N_\bullet)$ <a name="interval_cut"></a>


**Definition**. The interval cut morphism is a quasi-isomorphism described in $[1]$

**Implementation**. The class `Surjection_element` has the method `interval_cut` that models this map. 

MISSING SIGN

In [ ]:
n = 3
cup_2 = Surjection_element({(1,2,1,2):3}, torsion=2)
print(f'If cup_2 = {cup_2} then its action on {n}-simplices is given by \n' +
      f'IC(cup_2) = {cup_2.interval_cut(n)}')

## Power Operations <a name="power_operations"></a>

### Chain convention <a name="chain_convention"></a>

**Definition** We will work over $\mathbb Z/ p \mathbb Z$ where $p$ is a prime number. Let $\mathcal O$ be an $E_\infty$-operad and $A$ and $\mathcal O$-algebra. Let

\begin{equation} \label{equation: from W to O}
\iota : W \to \mathcal O(p)
\end{equation}

be  choice of $\mathbb F_p[\mathrm C_p]$-equivariant quasi-isomorphism. For $i \geq 0$,
\begin{equation*}
D_i : A \to A
\end{equation*}
is defined by
\begin{equation*}
D_i(a) = \iota(e_i)(a \otimes \cdots \otimes a).
\end{equation*}
If p = 2, define
\begin{equation*}
Q_s : A \to A
\end{equation*}
for $a \in A$ of degree $n$ by
\begin{equation*}
Q_s(a) = \begin{cases}
D_{s-n}(a) & s \geq d \\
0 & s < d
\end{cases}
\end{equation*}
and, if $p > 2$,
\begin{equation*}
Q_s : A \to A \qquad
\beta Q_s : A \to A
\end{equation*}
are respectively defined by
\begin{equation*}
Q_s(a) = \begin{cases}
D_{(2s-n)(p-1)}(a) & 2s \geq n \\
0 & 2s < n
\end{cases}
\qquad
\beta Q_s(a) = \begin{cases}
D_{(2s-n)(p-1)-1}(a) & 2s > n \\
0 & 2s \leq n.
\end{cases}
\end{equation*}
The power operations
\begin{equation*}
P_s : H_\bullet(A) \to H_{\bullet + s}(A)
\end{equation*}
for $p = 2$ and
\begin{equation*}
P_s : H_\bullet(A) \to H_{\bullet + 2s(p-1)}(A) \qquad
\beta P_s : H_\bullet(A) \to H_{\bullet + 2s(p-1) - 1}(A)
\end{equation*}

for $p > 2$, which are independent of $\iota$, are defined for a class $[a]$ of degree $n$ with cycle representative $a$ by

\begin{equation*}
P_s\big([a]\big) = \big[(-1)^s \nu(n) Q_s(a)\big] \qquad
\beta P_s\big([a]\big) = \big[(-1)^s \nu(n)\beta Q_s(a)\big]
\end{equation*}

where $\nu(n) = (-1)^{n(n-1)m/2}(m!)^n$ and $m = (p-1)/2$.

**Remark 2**. The use of $\nu$ is credited to Serre and is motivated by the identity $D_{q(p-1)}(a) = \nu(d)\, a$.

**Remark 3**. The notation $\beta P_s$ is motivated by the relationship of this operator and the Bockstein of the reduction $\mathbb Z \to \mathbb Z/p\mathbb Z$.

**Implementation**: We introduce a subclass of `object` modeling power operations at the chain level. More precisely, given a triple `p,s,n` of `int` we can initialize `Power_operation(p,s,n)`. The two methods `as_BarrattEccles_element` and `as_Surjection_element` are such that their outputs, say $x$ and $y$ respectively, satisfy 
\begin{equation*}
[x(a^{\otimes p})] = P_s([a]) \qquad
[y(a^{\otimes p})] = P_s([a])
\end{equation*}

where $a$ is a degree $n$ cycle in an algebra over the operad $\mathcal E$ or $Surj$ respectively with $\mathbb Z/p \mathbb Z$-coefficients.

When the optional boolean argument `bockstein` is `True`, the outputs $x$ and $y$ model $\beta P^s$, explicitly,

\begin{equation*}
[x(a^{\otimes p})] = \beta P_s([a]) \qquad
[y(a^{\otimes p})] = \beta P_s([a])
\end{equation*}
with $a$ as before.

In [ ]:
from clesto import PowerOperation

# Araki-Kudo operations (p == 2)
p, s, n = 2, 4, 2
op = PowerOperation(2,4,n)
print(f'The operation {str(op)} is represented \n' +
      f'in the Barratt-Eccles operad by \nx = {op.as_BarrattEccles_element()} \n' +
      f'and in the Surjection operad by \ny = {op.as_Surjection_element()}')

In [ ]:
# Dyer-Lashof operations (p > 2)
# Warning: computation speed is inversely correlated with (2s-n)(p-1)
p, s, n = 3, 3, 4
op = PowerOperation(p, s, n)
print(f'The operation {str(op)} with Z/{p}Z-coefficients is represented \n' +
      f'in the Barratt-Eccles operad by \nx = {op.as_BarrattEccles_element()} \n' +
      f'and in the Surjection operad by \ny = {op.as_Surjection_element()}')

### Cochain convention <a name="cochain_convention"></a>

## References <a name="references"></a>
